# Laptop Price Analytics:
Goal: Predict laptop prices and generate actionable insights for manufacturers and retailers—what features drive price, how brands position themselves, and how to price/bundle for margin and market share.


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os 
import streamlit as st
from sqlalchemy import create_engine 
from sklearn.model_selection import train_test_split, KFold, cross_val_score 
from sklearn.preprocessing import OneHotEncoder, StandardScaler 
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score 
from sklearn.linear_model import Ridge, Lasso 
from sklearn.ensemble import RandomForestRegressor 
from xgboost import XGBRegressor 
import shap

df = pd.read_csv(r'C:\Users\hp\Downloads\laptop_prices.csv')
df.head()

,Company,Product,TypeName,Inches,Ram,OS,Weight,Price_euros,Screen,ScreenW,...,RetinaDisplay,CPU_company,CPU_freq,CPU_model,PrimaryStorage,SecondaryStorage,PrimaryStorageType,SecondaryStorageType,GPU_company,GPU_model
0,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1339.69,Standard,2560,...,Yes,Intel,2.3,Core i5,128,0,SSD,No,Intel,Iris Plus Graphics 640
1,Apple,Macbook Air,Ultrabook,13.3,8,macOS,1.34,898.94,Standard,1440,...,No,Intel,1.8,Core i5,128,0,Flash Storage,No,Intel,HD Graphics 6000
2,HP,250 G6,Notebook,15.6,8,No OS,1.86,575.00,Full HD,1920,...,No,Intel,2.5,Core i5 7200U,256,0,SSD,No,Intel,HD Graphics 620
3,Apple,MacBook Pro,Ultrabook,15.4,16,macOS,1.83,2537.45,Standard,2880,...,Yes,Intel,2.7,Core i7,512,0,SSD,No,AMD,Radeon Pro 455
4,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1803.60,Standard,2560,...,Yes,Intel,3.1,Core i5,256,0,SSD,No,Intel,Iris Plus Graphics 650


In [3]:
df.shape
df.info()
df.isnull().sum()
df.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Company               1275 non-null   object 
 1   Product               1275 non-null   object 
 2   TypeName              1275 non-null   object 
 3   Inches                1275 non-null   float64
 4   Ram                   1275 non-null   int64  
 5   OS                    1275 non-null   object 
 6   Weight                1275 non-null   float64
 7   Price_euros           1275 non-null   float64
 8   Screen                1275 non-null   object 
 9   ScreenW               1275 non-null   int64  
 10  ScreenH               1275 non-null   int64  
 11  Touchscreen           1275 non-null   object 
 12  IPSpanel              1275 non-null   object 
 13  RetinaDisplay         1275 non-null   object 
 14  CPU_company           1275 non-null   object 
 15  CPU_freq             

0

## Data Cleaning:

In [5]:
# Current exchange rate (as of 29 Jan 2026) 
eur_to_inr = 109.99
# Convert price from euros to INR 
df['Price_inr'] = df['Price_euros'] * eur_to_inr 
# Optional: round to nearest rupee 
df['Price_inr'] = df['Price_inr'].round(0) 

In [6]:
df['Price_inr']= df['Price_inr'].astype('int64')
df.head()

,Company,Product,TypeName,Inches,Ram,OS,Weight,Price_euros,Screen,ScreenW,...,CPU_company,CPU_freq,CPU_model,PrimaryStorage,SecondaryStorage,PrimaryStorageType,SecondaryStorageType,GPU_company,GPU_model,Price_inr
0,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1339.69,Standard,2560,...,Intel,2.3,Core i5,128,0,SSD,No,Intel,Iris Plus Graphics 640,147353
1,Apple,Macbook Air,Ultrabook,13.3,8,macOS,1.34,898.94,Standard,1440,...,Intel,1.8,Core i5,128,0,Flash Storage,No,Intel,HD Graphics 6000,98874
2,HP,250 G6,Notebook,15.6,8,No OS,1.86,575.00,Full HD,1920,...,Intel,2.5,Core i5 7200U,256,0,SSD,No,Intel,HD Graphics 620,63244
3,Apple,MacBook Pro,Ultrabook,15.4,16,macOS,1.83,2537.45,Standard,2880,...,Intel,2.7,Core i7,512,0,SSD,No,AMD,Radeon Pro 455,279094
4,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1803.60,Standard,2560,...,Intel,3.1,Core i5,256,0,SSD,No,Intel,Iris Plus Graphics 650,198378


In [7]:
# Remove leading and trailing spaces from selected columns
for col in ['Touchscreen', 'IPSpanel', 'RetinaDisplay']:
    df[col] = df[col].astype(str).str.strip()

# Remove obvious duplicates 
df = df.drop_duplicates()

# Validate numeric columns
num_cols = ['Inches','Ram','Weight','Price_inr','ScreenW','ScreenH','CPU_freq','PrimaryStorage','SecondaryStorage']

df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

# Target variable must exist
df = df.dropna(subset=['Price_inr'])

# Remove extreme outliers (optional, keep for robustness)
q_low, q_high = df['Price_inr'].quantile([0.001, 0.999])

df = df[(df['Price_inr'] >= q_low) & (df['Price_inr'] <= q_high)]

# Create binary flags 
df['Touchscreen_Flag'] = (df['Touchscreen'].str.title() == 'Yes').astype(int) 
df['IPSpanel_Flag'] = (df['IPSpanel'].str.title() == 'Yes').astype(int) 
df['Retina_Flag'] = (df['RetinaDisplay'].str.title() == 'Yes').astype(int)


# Screen pixels and density proxy 
df['Pixels'] = df['ScreenW'] * df['ScreenH'] 
df['PPI_proxy'] = df['Pixels'] / (df['Inches']**2) 

# Storage totals 
df['TotalStorage'] = df['PrimaryStorage'].fillna(0) + df['SecondaryStorage'].fillna(0) 

# CPU/GPU simplifications 
df['CPU_brand'] = df['CPU_company'].str.strip().str.title() 
df['GPU_brand'] = df['GPU_company'].str.strip().str.title() 

# OS normalization 
df['OS_clean'] = (df['OS'].str.lower() 
				.replace({'mac os x':'macos','chrome os':'chrome os','windows 10 s':'windows 10 s'}) .str.title()) 
				
# TypeName normalization 
df['Type_clean'] = df['TypeName'].str.strip().str.title()

df.head()

,Company,Product,TypeName,Inches,Ram,OS,Weight,Price_euros,Screen,ScreenW,...,Touchscreen_Flag,IPSpanel_Flag,Retina_Flag,Pixels,PPI_proxy,TotalStorage,CPU_brand,GPU_brand,OS_clean,Type_clean
0,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1339.69,Standard,2560,...,0,1,1,4096000,23155.633445,128,Intel,Intel,Macos,Ultrabook
1,Apple,Macbook Air,Ultrabook,13.3,8,macOS,1.34,898.94,Standard,1440,...,0,0,0,1296000,7326.587145,128,Intel,Intel,Macos,Ultrabook
2,HP,250 G6,Notebook,15.6,8,No OS,1.86,575.00,Full HD,1920,...,0,0,0,2073600,8520.710059,256,Intel,Intel,No Os,Notebook
3,Apple,MacBook Pro,Ultrabook,15.4,16,macOS,1.83,2537.45,Standard,2880,...,0,1,1,5184000,21858.660820,512,Intel,Amd,Macos,Ultrabook
4,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1803.60,Standard,2560,...,0,1,1,4096000,23155.633445,256,Intel,Intel,Macos,Ultrabook


## Exploratory Data Analysis (EDA):

In [9]:
import seaborn as sns

sns.distplot(df['Price_inr'])

<Axes: xlabel='Price_inr', ylabel='Density'>

In [10]:
import matplotlib.pyplot as plt

# Count of laptops by TypeName
df['TypeName'].value_counts().plot(kind='bar', figsize=(5,4))

sns.barplot(x=df['Type_clean'], y=df['Price_inr'])
plt.title('Laptop Count by Type_clean')
plt.xlabel('TypeName')
plt.ylabel('Count')
plt.tick_params(axis='x', rotation=90)
plt.show()


In [11]:
# Distributions
fig, axes = plt.subplots(2, 3, figsize=(18,10))

#Plot 1: Price distribution
sns.histplot(df['Price_inr'], bins=40, ax=axes[0,0], color='steelblue')
axes[0,0].set_title('Price distribution')

# Plot 2: Price of laptops by TypeName
sns.barplot(x='Type_clean', y='Price_inr', data=df, ax=axes[0,1])
axes[0,1].set_title('Price by TypeName')
axes[0,1].tick_params(axis='x', rotation=45)

# Plot 3: laptop price by Company
sns.barplot(x='Company', y='Price_inr', data=df.sort_values('Company'), ax=axes[0,2])
axes[0,2].set_title('Price by Company')
axes[0,2].tick_params(axis='x', rotation=90)

# Plot 4: Price by CPU frequency
sns.scatterplot(x='CPU_freq', y='Price_inr', hue='CPU_brand', data=df, ax=axes[1,0])
axes[1,0].set_title('Price vs CPU frequency')

# Plot 5: Price by RAM
sns.scatterplot(x='Ram', y='Price_inr', hue='GPU_brand', data=df, ax=axes[1,1])
axes[1,1].set_title('Price vs RAM')

# Plot 6: Price by Screen definition
sns.boxplot(x='Screen', y='Price_inr', data=df, ax=axes[1,2])
axes[1,2].set_title('Price by Screen definition')
axes[1,2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [12]:
# Cell 6: Correlation heatmap (numeric)
num_for_corr = df[['Price_inr','Inches','Ram','Weight','ScreenW','ScreenH','CPU_freq','PrimaryStorage','SecondaryStorage','Pixels','PPI_proxy','TotalStorage']]
plt.figure(figsize=(10,8))
sns.heatmap(num_for_corr.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation heatmap')
plt.show()

In [13]:
# Segment comparisons
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Plot 1: Average price by OS
sns.barplot(x='OS_clean', y='Price_inr', data=df, ax=axes[0])
axes[0].set_title('Average price by OS')
axes[0].tick_params(axis='x', rotation=45)

# Plot 2: Touchscreen impact across screen definitions
sns.barplot(x='Touchscreen_Flag', y='Price_inr', hue='Screen', data=df)
axes[1].set_title('Touchscreen impact across screen definitions')

plt.tight_layout()
plt.show()

### Feature Engineering for Modeling:

In [15]:
# Feature set 
target = 'Price_inr'

categorical_cols = ['Company','Product','Type_clean','OS_clean','Screen','PrimaryStorageType','SecondaryStorageType','CPU_brand','GPU_brand','Touchscreen','IPSpanel','RetinaDisplay'] 
numeric_cols = ['Inches','Ram','Weight','ScreenW','ScreenH','CPU_freq','PrimaryStorage','SecondaryStorage','Pixels','PPI_proxy','TotalStorage','Touchscreen_Flag','IPSpanel_Flag','Retina_Flag']

# Drop high-cardinality 'Product' for baseline (we’ll add later in advanced model) 
categorical_cols_baseline = [c for c in categorical_cols if c != 'Product']

X = df[categorical_cols_baseline + numeric_cols] 
y = df[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Preprocessor 
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False) 
scaler = StandardScaler() 
preprocessor = ColumnTransformer(transformers=[('cat', ohe, categorical_cols_baseline),('num', scaler, numeric_cols)])

In [17]:
# Models 
ridge = Pipeline(steps=[('prep', preprocessor), ('model', Ridge(alpha=1.0, random_state=42))]) 
lasso = Pipeline(steps=[('prep', preprocessor), ('model', Lasso(alpha=0.001, random_state=42, max_iter=10000))]) 
rf = Pipeline(steps=[('prep', preprocessor), ('model', RandomForestRegressor(n_estimators=400, max_depth=None, random_state=42, n_jobs=-1))]) 
xgb = Pipeline(steps=[('prep', preprocessor), ('model', XGBRegressor(n_estimators=600, max_depth=6, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1))]) 
models = {'Ridge': ridge, 'Lasso': lasso, 'RandomForest': rf, 'XGBoost': xgb}

In [18]:
# Training and evaluation 
def evaluate_model(name, pipe): 
    pipe.fit(X_train, y_train) 
    preds = pipe.predict(X_test) 
    mae = mean_absolute_error(y_test, preds) 
    rmse = mean_squared_error(y_test, preds, squared=False) 
    r2 = r2_score(y_test, preds) 
    print(f"{name}: MAE={mae:.2f}, RMSE={rmse:.2f}, R2={r2:.3f}") 
    return {'name':name, 'mae':mae, 'rmse':rmse, 'r2':r2, 'model':pipe} 
    
results = [evaluate_model(n, m) for n, m in models.items()] 
best = sorted(results, key=lambda x: x['rmse'])[0] 
best['name'], best['r2']

Ridge: MAE=26964.57, RMSE=36564.51, R2=0.748
Lasso: MAE=26812.21, RMSE=36332.13, R2=0.751
RandomForest: MAE=19847.20, RMSE=31580.22, R2=0.812
XGBoost: MAE=18589.06, RMSE=28101.28, R2=0.851


('XGBoost', 0.8511995077133179)

In [19]:
# Get feature names after preprocessing 
prep = best['model'].named_steps['prep'] 
cat_features = prep.named_transformers_['cat'].get_feature_names_out(categorical_cols_baseline) 
num_features = np.array(numeric_cols) 
feature_names = np.concatenate([cat_features, num_features])

In [20]:
# Model-based importance 
if best['name'] == 'RandomForest': 
    importances = best['model'].named_steps['model'].feature_importances_ 
elif best['name'] == 'XGBoost': 
    importances = best['model'].named_steps['model'].feature_importances_ 
else: # Coefficients for linear models 
    importances = np.abs(best['model'].named_steps['model'].coef_)

fi = pd.DataFrame({'feature': feature_names, 'importance': importances}) 
fi = fi.sort_values('importance', ascending=False).head(25) 

plt.figure(figsize=(10,8)) 
sns.barplot(y='feature', x='importance', data=fi, palette='viridis') 
plt.title(f'Top 25 feature importance ({best["name"]})') 
plt.tight_layout() 
plt.show()

In [21]:
# SHAP (for tree-based models) 
# Use a small sample for speed 
X_sample = X_test.sample(n=min(300, len(X_test)), random_state=42) 
best['model'].fit(X_train, y_train) 
X_trans = prep.transform(X_sample)

if best['name'] in ['RandomForest','XGBoost']: 
    explainer = shap.TreeExplainer(best['model'].named_steps['model']) 
    shap_values = explainer.shap_values(X_trans) 
    shap.summary_plot(shap_values, features=X_trans, feature_names=feature_names, plot_type='bar', show=False) 
    plt.title('SHAP summary (bar)') 
    plt.show() 
    
    shap.summary_plot(shap_values, features=X_trans, feature_names=feature_names, show=False) 
    plt.title('SHAP summary (beeswarm)') 
    plt.show()

In [22]:
# Price bands 
bins = [0, 500, 1000, 1500, 2500, 10000] 
labels = ['Budget','Mid','Upper-Mid','Premium','Ultra'] 
df['PriceBand'] = pd.cut(df['Price_euros'], bins=bins, labels=labels, include_lowest=True)

# Brand share by band 
brand_band = (df.groupby(['Company','PriceBand']) .size() .reset_index(name='count')) 
total_band = brand_band.groupby('PriceBand')['count'].transform('sum') 
brand_band['share_in_band'] = brand_band['count'] / total_band

# Type positioning 
type_band = (df.groupby(['Type_clean','PriceBand']).agg(avg_price=('Price_euros','mean'),n=('Price_euros','size')) .reset_index())

In [23]:
# Visuals 
plt.figure(figsize=(12,6)) 
sns.barplot(x='PriceBand', y='share_in_band', hue='Company', data=brand_band[brand_band['share_in_band']>0.05]) 
plt.title('Brand share within price bands (top contributors)') 
plt.show() 

plt.figure(figsize=(12,6)) 
sns.scatterplot(x='PriceBand', y='avg_price', hue='Type_clean', size='n', data=type_band) 
plt.title('Type positioning across price bands') 
plt.show()

In [24]:
# Pricing what-if simulation 
# Example: simulate price change if RAM increases and storage switches to SSD
def simulate_price(row, ram_increase=8, ssd_bonus=1): 
    base_features = row.copy() 
    base_features['Ram'] = base_features['Ram'] + ram_increase
    if base_features['PrimaryStorageType'].lower() != 'ssd': 
        base_features['PrimaryStorageType'] = 'SSD'
    sim_row = pd.DataFrame([base_features])[X.columns] 
    pred = best['model'].predict(sim_row)[0] 
    return pred

sample_rows = df.sample(5, random_state=42) 
simulated_prices = [] 
for _, r in sample_rows.iterrows(): 
    simulated_prices.append({ 'Company': r['Company'], 'OriginalPrice': r['Price_inr'], 'SimulatedPrice': simulate_price(r) }) 
    pd.DataFrame(simulated_prices)

### Cross-Validation & Model Selection:

In [26]:
# K-Fold CV on best two models
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for name in ['RandomForest','XGBoost']:
    pipe = models[name]
    scores = cross_val_score(pipe, X, y, scoring='neg_root_mean_squared_error', cv=kf, n_jobs=-1)
    print(f"{name} CV RMSE: {(-scores).mean():.2f} ± {(-scores).std():.2f}")

RandomForest CV RMSE: 31364.04 ± 3773.84
XGBoost CV RMSE: 28580.22 ± 2204.08


In [27]:
# save model via joblib
import joblib
joblib.dump(best['model'], f"best_model_{best['name']}.joblib")

['best_model_XGBoost.joblib']

In [28]:
# Save cleaned data and feature importance 
df.to_csv('cleaned_laptop_prices.csv', index=False) 
fi.to_csv('feature_importance.csv', index=False)

### Loading tha Cleaned Dataset in Mysql:

In [36]:
username = "root"
password = "MysqlSunita80"
host = "localhost"
port = "3306"
database = "Laptop_pricing"

engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
)
table_name='laptop_prices'
df.to_sql(table_name, engine, if_exists='replace', index=False)
print(f"data succsessfully inserted into the '{table_name}' table in the '{database}' database.")

data succsessfully inserted into the 'laptop_prices' table in the 'Laptop_pricing' database.
